In [1]:
from libs.GraphToolbox import GraphManager
from libs.InvertedIndexToolbox import getNGrams
import operator
import pprint
import os
import random
import numpy as np

In [2]:
index_properties = [str("<http://rdata2graph.sap.com/hilti_erp/property/MARA_FERT.MAKTX>").lower(), str("<http://rdata2graph.sap.com/hilti_web/property/Products.name>").lower()]
src_graphmanager = GraphManager()
src_graphmanager.setIndexProperties(index_properties)
src_graphmanager.readGraphFromNTFile("C:/Users/D072202/RData2Graph/rdata2graph/data/sap_hilti_data/graph_triples_hilti_erp.nt")
tgt_graphmanager = GraphManager()
tgt_graphmanager.setIndexProperties(index_properties)
tgt_graphmanager.readGraphFromNTFile("C:/Users/D072202/RData2Graph/rdata2graph/data/sap_hilti_data/graph_triples_hilti_web.nt")
print("Done")

Done


In [3]:
randomly_drawn_nodes = list()
N=2000
for nodeid in src_graphmanager.graph.keys():
    if random.random() > 1-N/len(src_graphmanager.graph.keys()):
        randomly_drawn_nodes.append(nodeid)
print('Selected ' + str(len(randomly_drawn_nodes)) + ' samples out of ' + str(len(src_graphmanager.graph.keys())))

Selected 2005 samples out of 12297


In [4]:
def sortkey(val):
    return val[1]

def aggregate_to_dict(indices):
    tmp_tgt_ind = dict()
    for index in indices:
            if index in tmp_tgt_ind.keys():
                tmp_tgt_ind[index] = tmp_tgt_ind[index] + 1
            else:
                tmp_tgt_ind[index] = 1
    return tmp_tgt_ind


In [5]:
f = open('negative_sampled_gold_standard', 'w+')

for nodeid in randomly_drawn_nodes:
    indices = []
    for src_prop in src_graphmanager.indices.keys():
                for tgt_prop in tgt_graphmanager.indices.keys():
                    try:
                        indices = indices + tgt_graphmanager.indices[tgt_prop].getIndicesForValue(src_graphmanager.graph[nodeid][src_prop])
                    except KeyError:
                        pass
    tmp_tgt_ind = aggregate_to_dict(indices)

    best_matching_resources = list(tmp_tgt_ind.items())
    if best_matching_resources is None or best_matching_resources == []:
        continue
    best_matching_resources.sort(key = sortkey, reverse = True)
    best_matching_resources = np.array(best_matching_resources)
    current_correspondences = list()
    current_correspondences = current_correspondences + best_matching_resources[:3,0].tolist()
    current_correspondences = current_correspondences + best_matching_resources[len(best_matching_resources)-3:len(best_matching_resources),0].tolist()  
    for correspondence in current_correspondences:
        f.write(nodeid.replace("<","").replace(">","") + "\t" + correspondence.replace("<","").replace(">","") +"\n")
f.close()
print("Done")

Done
